In [0]:
!pip install yfinance

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

start=datetime.now()-timedelta(days=730)
end=datetime.now()
USDINR=yf.download("USDINR=X",start=start,end=end)
USDINR=USDINR.reset_index()
DXY=yf.download("DX-Y.NYB",start=start,end=end)
DXY=DXY.reset_index()
DXY=DXY[['Date','Close']].copy()
DXY.columns = [col[0] if isinstance(col, tuple) else col for col in DXY.columns]
DXY.rename(columns={'Close':'DXYClose'},inplace=True)
USDINR_Filtered=USDINR[['Date','Close']].copy()
USDINR_Filtered.columns = [col[0] if isinstance(col, tuple) else col for col in USDINR_Filtered.columns]
USDINR_Filtered.rename(columns={'Close':'USDINRClose'},inplace=True)
print(USDINR_Filtered.sort_values(by='Date',ascending=False).head(5))
merge=pd.merge(USDINR_Filtered,DXY,on='Date',how='outer')
merge=merge.sort_values(by='Date')
merge=merge.ffill()
merged=spark.createDataFrame(merge)
merged.write.mode("overwrite").format('delta').saveAsTable("USDINR_DXY_")
#DXY=spark.createDataFrame(DXY)
#DXY.write.mode("overwrite").format('delta').saveAsTable("DXY_")

In [0]:
%sql
select * from USDINR_DXY_
order by Date desc
limit 10;

In [0]:
spark.sql("SHOW TABLES IN default").show()